Import neccessary libraries

In [28]:
# -*- coding: utf-8 -*-


import numpy as np 
import pandas as pd
from pandas_datareader import data, wb
import matplotlib.pyplot as plt 
import math
from datetime import datetime
%matplotlib inline








Data preparation

In [29]:
csv_f = pd.read_csv('L_T_data.csv')
data = pd.DataFrame(csv_f)
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace = True)
df = data[~data.index.duplicated(keep = 'first')]
df1 = df[:-1]
dfMajor = df1.loc[:,'BTC': 'LTC']
dfMinor = df1.loc[:,'ETC':]

Technical Analysis prep, calc and weights

In [30]:

#Momentum prep and calc 
mom_1_var = 15
mom_1 = df1.pct_change(mom_1_var)
mom_2_var = 30
mom_2 = df1.pct_change(mom_2_var)
    
#Sortino prep and calc
period = 30
Rtn = df1.pct_change(period)
dailyRtn = df1.pct_change(1)
averageofP = dailyRtn.rolling(window=period).mean()
downsideDev = np.where(dailyRtn < averageofP, dailyRtn, np.NaN)
downsideDev_df = pd.DataFrame(downsideDev, index=df1.index, columns=df1.columns).reset_index()
downsideDev_df = downsideDev_df.set_index("date")
std_dd_df = downsideDev_df.rolling(window=period, min_periods = 1).std()
sortino_R = Rtn / std_dd_df
period_Sort = 15
sortino_Avg = sortino_R.rolling(window=period_Sort).mean()
sortino_mom_var = 20
sortino_mom = sortino_Avg.pct_change(period_Sort)
    
#Technical weights
m_weight_1 = 0.5
m_weight_2 = 0.5
r_Weight_1 = 0.5
r_Weight_2 = 0.5
mom_1_R = mom_1.rank(1, ascending = False)
mom_2_R = mom_2.rank(1, ascending = False )
mom_R = (mom_1_R * m_weight_1) + (mom_2_R * m_weight_2)
sortino_R = sortino_mom.rank(1, ascending = False)
total_Rank = (sortino_R *r_Weight_1) + (mom_R* r_Weight_2)
total_Rank = total_Rank.fillna(1000)


Fundamental analysis

In [31]:
fundamental_data = pd.read_csv('Fundamentals.csv')
df_f = pd.DataFrame(fundamental_data)
df_f['date'] = pd.to_datetime(df_f['date'])
df_f.set_index('date', inplace = True)


Create groups (Major/Minor)

In [32]:
TRmajor = total_Rank.loc[:,'BTC':'LTC']
TRminor = total_Rank.loc[:,'ETC':]
TRmajor = TRmajor.rank(1, ascending = True)
TRminor = TRminor.rank(1, ascending = True)


Major Prep + weights

In [33]:
df_f_Mj = df_f.loc[:,'BTC':'LTC']
df_f_Mj.tail()
df_t_Mj = TRmajor[-1:]
df_ft_Mj = (df_f_Mj*0.6) + (df_t_Mj*0.4)
df_ft_Mj = df_ft_Mj.rank(1, ascending = True)

Minor Prep + weights

In [34]:
df_f_Mn = df_f.loc[:,'ETC':'ADA']
df_f_Mn = df_f_Mn.rank(1, ascending = True)
df_t_Mn = TRminor[-1:]
df_ft_Mn = (df_f_Mn*0.6) + (df_t_Mn*0.4)
df_ft_Mn = df_ft_Mn.rank(1, ascending = True)

Major ranking 

In [35]:
Mj_rank_input = 4
Mj_rank = np.where(df_ft_Mj < Mj_rank_input, 1, 0)
Mj_rank = pd.DataFrame(Mj_rank, index = df_ft_Mn.index, columns = df_ft_Mj.columns).reset_index()
Mj_rank = Mj_rank.set_index("date")
mask = Mj_rank.ne(0)
Mj_rank = Mj_rank.loc[mask.any(1), mask.any()]

Minor ranking

In [36]:
Mn_rank_input = 5
Mn_rank = np.where(df_ft_Mn < Mn_rank_input, 1, 0)
Mn_rank = pd.DataFrame(Mn_rank, index = df_ft_Mn.index, columns = df_ft_Mn.columns).reset_index()
Mn_rank = Mn_rank.set_index("date")
mask = Mn_rank.ne(0)
Mn_rank = Mn_rank.loc[mask.any(1), mask.any()]



Weights Major and Minor

In [37]:
Mj_weights = Mj_rank / (Mj_rank_input -1)
Mn_weights = Mn_rank/ (Mn_rank_input -1)

Major_W = 0.5
Minor_W = 1 - Major_W

Mj_weights = Mj_weights * Major_W
Mn_weights = Mn_weights * Minor_W


Portfolio 

In [38]:
portfolio = pd.concat([Mj_weights, Mn_weights], axis = 1)
#append(Mn_weights, ignore_index = True)- this method put them on seperate rows, but concat worked
portfolio

,BTC,ETH,XRP,SC,XLM,NEO,EOS
date,,,,,,,
2017-12-29,0.166667,0.166667,0.166667,0.125,0.125,0.125,0.125
